In [28]:
# ============================================================================
# CELL 1: Install Required Packages
# ============================================================================
# Run this cell first to install all necessary packages
# This may take a few minutes

!pip install azure-ai-projects
!pip install azure-identity
!pip install azure-ai-agents
!pip install python-dotenv

print("✅ Packages installed successfully!")



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
✅ Packages installed successfully!


In [29]:
# ============================================================================
# CELL 2: Import Required Libraries
# ============================================================================
# Import all the libraries we'll need for this notebook

import os
import json
from datetime import datetime
from dotenv import load_dotenv

# Azure AI imports
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient

print("✅ Libraries imported successfully!")


# Load environment variables from .env file in the current directory
load_dotenv()

print("✅ .env file loaded!")

✅ Libraries imported successfully!
✅ .env file loaded!


In [30]:
# ============================================================================
# CELL 3: Set Up Azure Credentials and Configuration
# ============================================================================
# Configure your Azure AI Foundry project details here
# You can find these in your Azure AI Foundry portal

# Load project_endpoint and model_deployment_name from environment variables

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

print(f"Project Endpoint from .env: {project_endpoint}")
print(f"Model Deployment Name from .env: {model_deployment_name}")

Project Endpoint from .env: https://aifoundryaml0058444754.services.ai.azure.com/api/projects/aifoundryaml0058444754-project
Model Deployment Name from .env: gpt-4o-mini


In [31]:
# ============================================================================
# CELL 4: Authenticate with Azure
# ============================================================================
# Create credential object for authentication
# This will use your Azure CLI login (run 'az login' in terminal first)

credential = DefaultAzureCredential()
print("✅ Authentication credential created!")
print("   Make sure you've run 'az login' in your terminal")

✅ Authentication credential created!
   Make sure you've run 'az login' in your terminal


In [32]:
# ============================================================================
# CELL 5: Create Project Client
# ============================================================================
# Connect to your Azure AI Foundry project

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=credential
)

print("✅ Project client created successfully!")

✅ Project client created successfully!


In [33]:
# ============================================================================
# CELL 7: Define Adaptive Card Templates
# ============================================================================
# Create reusable Adaptive Card templates as JSON objects
# These cards can be returned by the agent

# Template 1: Welcome Card
welcome_card = {
    "type": "AdaptiveCard",
    "version": "1.5",
    "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
    "body": [
        {
            "type": "TextBlock",
            "text": "🎉 Welcome to Azure AI Agent!",
            "weight": "Bolder",
            "size": "Large",
            "color": "Accent"
        },
        {
            "type": "TextBlock",
            "text": "I'm your AI assistant powered by Azure AI Foundry. I can help you with various tasks and display rich, interactive cards.",
            "wrap": True
        },
        {
            "type": "FactSet",
            "facts": [
                {"title": "Status:", "value": "Online"},
                {"title": "Model:", "value": model_deployment_name},
                {"title": "Time:", "value": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
            ]
        }
    ]
}

# Template 2: Task Card
task_card_template = {
    "type": "AdaptiveCard",
    "version": "1.5",
    "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
    "body": [
        {
            "type": "TextBlock",
            "text": "📋 Task Information",
            "weight": "Bolder",
            "size": "Large"
        },
        {
            "type": "Input.Text",
            "id": "taskTitle",
            "placeholder": "Enter task title",
            "label": "Task Title"
        },
        {
            "type": "Input.Text",
            "id": "taskDescription",
            "placeholder": "Enter task description",
            "label": "Description",
            "isMultiline": True
        },
        {
            "type": "Input.ChoiceSet",
            "id": "priority",
            "label": "Priority",
            "choices": [
                {"title": "High", "value": "high"},
                {"title": "Medium", "value": "medium"},
                {"title": "Low", "value": "low"}
            ],
            "value": "medium"
        },
        {
            "type": "Input.Date",
            "id": "dueDate",
            "label": "Due Date"
        }
    ],
    "actions": [
        {
            "type": "Action.Submit",
            "title": "Create Task",
            "data": {"action": "create_task"}
        }
    ]
}

# Template 3: User Profile Card
profile_card_template = {
    "type": "AdaptiveCard",
    "version": "1.5",
    "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
    "body": [
        {
            "type": "ColumnSet",
            "columns": [
                {
                    "type": "Column",
                    "width": "auto",
                    "items": [
                        {
                            "type": "Image",
                            "url": "https://via.placeholder.com/80",
                            "style": "Person",
                            "size": "Medium"
                        }
                    ]
                },
                {
                    "type": "Column",
                    "width": "stretch",
                    "items": [
                        {
                            "type": "TextBlock",
                            "text": "Azure User",
                            "weight": "Bolder",
                            "size": "Large"
                        },
                        {
                            "type": "TextBlock",
                            "text": "AI Developer",
                            "isSubtle": True
                        }
                    ]
                }
            ]
        },
        {
            "type": "FactSet",
            "facts": [
                {"title": "Email:", "value": "user@example.com"},
                {"title": "Department:", "value": "AI Development"},
                {"title": "Location:", "value": "Cloud"}
            ]
        }
    ]
}

print("✅ Adaptive Card templates created!")
print(f"   - Welcome Card")
print(f"   - Task Card")
print(f"   - Profile Card")

✅ Adaptive Card templates created!
   - Welcome Card
   - Task Card
   - Profile Card


In [34]:
# ============================================================================
# CELL 8: Create a Function to Format Cards for Agent Response
# ============================================================================
# This function formats adaptive cards to be included in agent responses

def format_card_for_agent(card_data, title="Adaptive Card"):
    """
    Format an adaptive card to be included in the agent's response
    
    Args:
        card_data: Dictionary containing the adaptive card JSON
        title: Title to display above the card
    
    Returns:
        Formatted string with the card data
    """
    card_json = json.dumps(card_data, indent=2)
    
    formatted_response = f"""
Here's a {title}:

```json
{card_json}
```

This card can be rendered in applications that support Adaptive Cards.
"""
    
    return formatted_response

print("✅ Card formatting function created!")

✅ Card formatting function created!


In [35]:
# ============================================================================
# CELL 9: Create Agent Instructions with Card Examples
# ============================================================================
# Define instructions for the agent that include information about adaptive cards

agent_instructions = f"""
You are a helpful AI assistant powered by Azure AI Foundry. 

Your capabilities include:
1. Answering questions about various topics
2. Providing information in Adaptive Card format when requested
3. Creating structured, interactive cards for better user experience

When a user asks for a card or structured information, you can provide:
- Welcome cards
- Task creation cards
- Profile cards
- Custom cards based on user needs

Available card types:
- "welcome" or "greeting" → Show welcome card
- "task" or "create task" → Show task creation card
- "profile" or "user info" → Show user profile card

Always be helpful, clear, and provide rich formatted responses when appropriate.
Current model: {model_deployment_name}
"""

print("✅ Agent instructions defined!")
print("\nInstructions preview:")
print(agent_instructions[:200] + "...")

✅ Agent instructions defined!

Instructions preview:

You are a helpful AI assistant powered by Azure AI Foundry. 

Your capabilities include:
1. Answering questions about various topics
2. Providing information in Adaptive Card format when requested
3....


In [36]:
# ============================================================================
# CELL 10: Create the Agent
# ============================================================================
# Create an agent in Azure AI Foundry with our instructions

print("Creating agent in Azure AI Foundry...")

agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="my-agent",
    instructions="You are a helpful agent"
)
# Store agent ID for later use
agent_id = agent.id

print(f"✅ Agent created successfully!")
print(f"   Agent ID: {agent_id}")
print(f"   Agent Name: {agent.name}")
print(f"   Model: {agent.model}")

Creating agent in Azure AI Foundry...


✅ Agent created successfully!
   Agent ID: asst_cNh7I6G9xKsPSE4fA8eNifoR
   Agent Name: my-agent
   Model: gpt-4o-mini


In [37]:
# ============================================================================
# CELL 11: Create a Thread for Conversation
# ============================================================================
# Create a conversation thread where we'll interact with the agent

print("Creating conversation thread...")

thread = project_client.agents.threads.create()
thread_id = thread.id

print(f"✅ Thread created successfully!")
print(f"   Thread ID: {thread_id}")

Creating conversation thread...


✅ Thread created successfully!
   Thread ID: thread_hNe9UPzItBovSM68JeqPt1ty


In [38]:
# ============================================================================
# CELL 12: Define Helper Function to Chat with Agent (FINAL FIX)
# ============================================================================
# Create a reusable function to send messages and get responses

def chat_with_agent(message, return_card=False):
    """
    Send a message to the agent and get a response
    
    Args:
        message: The message to send to the agent
        return_card: If True, check if response should include a card
    
    Returns:
        The agent's response text and optionally a card
    """
    print(f"\n💬 You: {message}")
    print("🤔 Agent is thinking...")
    
    # Create a message in the thread
    message_obj = project_client.agents.messages.create(
        thread_id=thread_id,
        role="user",
        content=message
    )
    
    # Run the agent
    run = project_client.agents.runs.create(
        thread_id=thread_id,
        agent_id=agent_id
    )
    
    # Wait for completion
    import time
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.runs.get(
            thread_id=thread_id, 
            run_id=run.id
        )
    
    # Check if run was successful
    if run.status == "completed":
        # Get messages - this returns an ItemPaged object
        messages = project_client.agents.messages.list(
            thread_id=thread_id
        )
        
        # Iterate directly over messages (no .data attribute needed)
        for msg in messages:  # ✅ CHANGED: removed .data
            if msg.role == "assistant":
                # Handle message content safely
                if hasattr(msg, 'content') and len(msg.content) > 0:
                    if hasattr(msg.content[0], 'text'):
                        response_text = msg.content[0].text.value
                    else:
                        response_text = str(msg.content[0])
                else:
                    response_text = "No response content"
                
                print(f"🤖 Agent: {response_text}")
                
                # Check if we should return a card based on keywords
                if return_card:
                    message_lower = message.lower()
                    if "welcome" in message_lower or "greeting" in message_lower:
                        return response_text, welcome_card
                    elif "task" in message_lower:
                        return response_text, task_card_template
                    elif "profile" in message_lower:
                        return response_text, profile_card_template
                
                return response_text, None
    else:
        print(f"❌ Run failed with status: {run.status}")
        return None, None

print("✅ Chat function defined!")

✅ Chat function defined!


In [39]:
# ============================================================================
# CELL 13: Example 1 - Simple Conversation
# ============================================================================
# Test basic conversation with the agent

print("\n" + "="*60)
print("EXAMPLE 1: Simple Conversation")
print("="*60)

response, card = chat_with_agent("Hello! What can you help me with?")



EXAMPLE 1: Simple Conversation

💬 You: Hello! What can you help me with?
🤔 Agent is thinking...


🤖 Agent: Hello! I can assist you with a variety of topics, including but not limited to:

- Answering questions about general knowledge and various subjects
- Providing explanations or summaries of concepts
- Assisting with writing or editing text, including essays and reports
- Offering language translation or improving language skills
- Helping with problem-solving in math and science
- Providing information or advice on technology
- Giving recommendations for books, movies, or other media

Let me know what you need help with!


In [40]:
# ============================================================================
# CELL 14: Example 2 - Request Welcome Card
# ============================================================================
# Request a welcome card from the agent

print("\n" + "="*60)
print("EXAMPLE 2: Request Welcome Card")
print("="*60)

response, card = chat_with_agent("Show me a welcome card", return_card=True)

if card:
    print("\n📋 Adaptive Card Generated:")
    print(json.dumps(card, indent=2))



EXAMPLE 2: Request Welcome Card

💬 You: Show me a welcome card
🤔 Agent is thinking...
🤖 Agent: I can't create images directly, but I can help you design a welcome card through a description! Here’s how you could layout a welcoming card:

### Front of the Card:
- **Background:** Soft pastel colors (e.g., light blue, lavender, or mint green)
- **Main Design Element:** An illustration of flowers or balloons
- **Text:** In elegant, cursive font - "Welcome!"

### Inside of the Card:
- **Greeting:** "We are so happy to have you here!"
- **Message:** "May your journey be filled with joy and wonderful experiences."
- **Closing:** "With warm regards, [Your Name or Organization]"

### Additional Elements:
- You can add small decorations like stars or hearts around the text or borders.

If you're looking to create a digital version, you might consider using graphic design tools like Canva or Adobe Spark, where you can easily find templates and customize them. Let me know if you want more specifi

In [41]:
# ============================================================================
# CELL 15: Example 3 - Request Task Card
# ============================================================================
# Request a task creation card

print("\n" + "="*60)
print("EXAMPLE 3: Request Task Card")
print("="*60)

response, card = chat_with_agent("I need to create a task", return_card=True)

if card:
    print("\n📋 Adaptive Card Generated:")
    print(json.dumps(card, indent=2))



EXAMPLE 3: Request Task Card

💬 You: I need to create a task
🤔 Agent is thinking...
🤖 Agent: Sure! To help you create a task, I'll need some details. Please provide the following information:

1. **Task Title:** What is the name of the task?
2. **Description:** What is the task about? Provide any necessary details.
3. **Due Date:** When does this task need to be completed?
4. **Priority Level:** Is this task high, medium, or low priority?
5. **Assigned To:** Who is responsible for completing this task?

Feel free to give me any additional details or context, and I’ll help you formulate it!

📋 Adaptive Card Generated:
{
  "type": "AdaptiveCard",
  "version": "1.5",
  "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
  "body": [
    {
      "type": "TextBlock",
      "text": "\ud83d\udccb Task Information",
      "weight": "Bolder",
      "size": "Large"
    },
    {
      "type": "Input.Text",
      "id": "taskTitle",
      "placeholder": "Enter task title",
      "label

In [42]:
# ============================================================================
# CELL 16: Example 4 - Request Profile Card
# ============================================================================
# Request a user profile card

print("\n" + "="*60)
print("EXAMPLE 4: Request Profile Card")
print("="*60)

response, card = chat_with_agent("Show me a user profile", return_card=True)

if card:
    print("\n📋 Adaptive Card Generated:")
    print(json.dumps(card, indent=2))






EXAMPLE 4: Request Profile Card

💬 You: Show me a user profile
🤔 Agent is thinking...
🤖 Agent: Sure! Here's an example of how a user profile could be structured. You can customize it according to your needs.

---

### User Profile

**Name:** Jane Doe  
**Username:** jane_doe92  
**Email:** jane.doe@example.com  
**Profile Picture:** [Insert Profile Picture Here]

**Bio:**  
A passionate graphic designer with over 5 years of experience in creating engaging visual content. Lover of art, travel, and photography.

**Location:**  
New York, NY, USA

**Skills:**  
- Graphic Design  
- Adobe Creative Suite (Photoshop, Illustrator, InDesign)  
- UI/UX Design  
- Branding and Identity  

**Interests:**  
- Digital Art  
- Photography  
- Travel  
- Cooking  

**Social Media Links:**  
- LinkedIn: [linkedin.com/in/janedoe](#)  
- Instagram: [instagram.com/janedoeart](#)

**Projects:**  
1. **Logo Design for Tech Startup** - Created a modern logo for a new tech company that reflects their innova

In [43]:
# ============================================================================
# CELL 17: Example 5 - Custom Dynamic Card
# ============================================================================
# Create a custom card with dynamic data

print("\n" + "="*60)
print("EXAMPLE 5: Create Custom Dynamic Card")
print("="*60)

# Get current weather info (simulated)
current_temp = 72
current_condition = "Partly Cloudy"
current_location = "Seattle"

# Create weather card
weather_card = {
    "type": "AdaptiveCard",
    "version": "1.5",
    "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
    "body": [
        {
            "type": "TextBlock",
            "text": "🌤️ Weather Update",
            "weight": "Bolder",
            "size": "Large"
        },
        {
            "type": "ColumnSet",
            "columns": [
                {
                    "type": "Column",
                    "width": "auto",
                    "items": [
                        {
                            "type": "TextBlock",
                            "text": f"{current_temp}°F",
                            "size": "ExtraLarge",
                            "weight": "Bolder"
                        }
                    ]
                },
                {
                    "type": "Column",
                    "width": "stretch",
                    "items": [
                        {
                            "type": "TextBlock",
                            "text": current_location,
                            "weight": "Bolder"
                        },
                        {
                            "type": "TextBlock",
                            "text": current_condition,
                            "isSubtle": True
                        }
                    ]
                }
            ]
        },
        {
            "type": "TextBlock",
            "text": f"Last updated: {datetime.now().strftime('%I:%M %p')}",
            "isSubtle": True,
            "size": "Small"
        }
    ]
}

print(f"Weather for {current_location}: {current_temp}°F, {current_condition}")
print("\n📋 Weather Card:")
print(json.dumps(weather_card, indent=2))


EXAMPLE 5: Create Custom Dynamic Card
Weather for Seattle: 72°F, Partly Cloudy

📋 Weather Card:
{
  "type": "AdaptiveCard",
  "version": "1.5",
  "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
  "body": [
    {
      "type": "TextBlock",
      "text": "\ud83c\udf24\ufe0f Weather Update",
      "weight": "Bolder",
      "size": "Large"
    },
    {
      "type": "ColumnSet",
      "columns": [
        {
          "type": "Column",
          "width": "auto",
          "items": [
            {
              "type": "TextBlock",
              "text": "72\u00b0F",
              "size": "ExtraLarge",
              "weight": "Bolder"
            }
          ]
        },
        {
          "type": "Column",
          "width": "stretch",
          "items": [
            {
              "type": "TextBlock",
              "text": "Seattle",
              "weight": "Bolder"
            },
            {
              "type": "TextBlock",
              "text": "Partly Cloudy",
  

In [44]:
# ============================================================================
# CELL 18: Visualize Card in Notebook (Optional)
# ============================================================================
# Display the adaptive card JSON in a formatted way

from IPython.display import JSON, display

print("\n" + "="*60)
print("VISUALIZING ADAPTIVE CARDS")
print("="*60)

print("\nWelcome Card Structure:")
display(JSON(welcome_card))

print("\nTask Card Structure:")
display(JSON(task_card_template))



VISUALIZING ADAPTIVE CARDS

Welcome Card Structure:


<IPython.core.display.JSON object>


Task Card Structure:


<IPython.core.display.JSON object>

In [45]:
# ============================================================================
# CELL 19: Save Cards to Files
# ============================================================================
# Save the generated cards to JSON files for later use

# Create output directory
os.makedirs("adaptive_cards_output", exist_ok=True)

# Save cards
cards_to_save = {
    "welcome_card.json": welcome_card,
    "task_card.json": task_card_template,
    "profile_card.json": profile_card_template,
    "weather_card.json": weather_card
}

for filename, card_data in cards_to_save.items():
    filepath = os.path.join("adaptive_cards_output", filename)
    with open(filepath, "w") as f:
        json.dump(card_data, f, indent=2)
    print(f"✅ Saved: {filepath}")


✅ Saved: adaptive_cards_output/welcome_card.json
✅ Saved: adaptive_cards_output/task_card.json
✅ Saved: adaptive_cards_output/profile_card.json
✅ Saved: adaptive_cards_output/weather_card.json


In [46]:
# ============================================================================
# CELL 20: Clean Up Resources (Optional)
# ============================================================================
# Clean up the agent and thread when you're done

print("\n" + "="*60)
print("CLEANUP (Run when finished)")
print("="*60)

# Uncomment these lines when you want to delete the agent and thread
# print("Deleting agent...")
# agents_client.delete_agent(agent_id)
# print("✅ Agent deleted")

# Note: Threads are automatically cleaned up after inactivity

print("✅ All examples completed successfully!")
print("\nNext steps:")
print("1. Test the cards in Adaptive Cards Designer: https://adaptivecards.io/designer/")
print("2. Integrate the cards into your application")
print("3. Customize the card templates for your use case")



CLEANUP (Run when finished)
✅ All examples completed successfully!

Next steps:
1. Test the cards in Adaptive Cards Designer: https://adaptivecards.io/designer/
2. Integrate the cards into your application
3. Customize the card templates for your use case
